In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import lsst.sims.photUtils as photUtils

In [2]:
import bandpassUtils as bu
import calcM5 as m5

In [3]:
defaultDirs = bu.setDefaultDirs()
addLosses = True
atmosphere = bu.readAtmosphere(defaultDirs['atmosphere'], atmosFile='atmos_10_aerosol.dat')
hardware, system = bu.buildHardwareAndSystem(defaultDirs, addLosses, atmosphereOverride=atmosphere)
atmosphere12 = bu.readAtmosphere(defaultDirs['atmosphere'], atmosFile='pachonModtranAtm_12_aerosol.dat')
hardware2, system2 = bu.buildHardwareAndSystem(defaultDirs, addLosses, atmosphereOverride=atmosphere12)

In [4]:
eff_wavelen = m5.get_effwavelens(system)

In [5]:
%%timeit
for airmass in (1.0, 1.1, 1.2):
    fwhm_eff, fwhm_geom = m5.scale_seeing(0.6, eff_wavelen, airmass=airmass)

35.9 µs ± 359 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [6]:
%%timeit
airmass = np.array([1.0, 1.1, 1.2])
fwhm_eff, fwhm_geom = m5.scale_seeing(0.6, eff_wavelen, airmass)

22 µs ± 83.6 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [5]:
airmass = np.array([1.0, 1.1, 1.2])
fwhm_eff, fwhm_geom = m5.scale_seeing(0.62, eff_wavelen, airmass)
print('X=1.0, each filter', np.array2string(fwhm_eff[:,0], precision=2))
defaults = photUtils.LSSTdefaults()
print(defaults.FWHMeff('u'), defaults.FWHMeff('g'), defaults.FWHMeff('r'), defaults.FWHMeff('i'), 
      defaults.FWHMeff('z'), defaults.FWHMeff('y'))
print('g band, all airmasses', fwhm_eff[1])

X=1.0, each filter [ 0.93  0.87  0.83  0.8   0.78  0.76]
0.92 0.87 0.83 0.8 0.78 0.76
g band, all airmasses [ 0.87463518  0.92610996  0.97574355]


In [6]:
t2 = m5.calcM5(hardware, system, atmosphere, X=1.0, return_t2_values=True)

Filter FWHMeff FWHMgeom SkyMag SkyCounts Zp_t Tb Sb kAtm Gamma Cm dCm_infinity m5 SourceCounts
u 0.93 0.82 22.95 50.2 26.50 0.022 0.036 0.5041 0.038079 22.74 0.75 23.40 520.63
g 0.87 0.77 22.24 387.6 28.30 0.118 0.143 0.2097 0.038724 24.38 0.19 24.76 783.45
r 0.83 0.73 21.20 796.2 28.13 0.100 0.113 0.1263 0.038977 24.43 0.10 24.34 977.63
i 0.80 0.71 20.47 1108.1 27.79 0.074 0.080 0.0962 0.039075 24.30 0.07 23.90 1081.30
z 0.78 0.69 19.60 1687.9 27.40 0.051 0.055 0.0685 0.039211 24.15 0.05 23.34 1267.38
y 0.76 0.68 18.63 2140.8 26.58 0.024 0.029 0.1760 0.039277 23.70 0.04 22.43 1383.53


In [7]:
t2_12 = m5.calcM5(hardware2, system2, atmosphere12, X=1.2, return_t2_values=True)

Filter FWHMeff FWHMgeom SkyMag SkyCounts Zp_t Tb Sb kAtm Gamma Cm dCm_infinity m5 SourceCounts
u 1.04 0.90 22.95 50.2 26.40 0.021 0.036 0.6033 0.038273 22.76 0.75 23.19 579.20
g 0.98 0.85 22.24 387.6 28.26 0.114 0.143 0.2512 0.038854 24.39 0.19 24.60 872.41
r 0.92 0.81 21.20 796.2 28.10 0.098 0.113 0.1511 0.039082 24.43 0.10 24.20 1089.12
i 0.89 0.78 20.47 1108.1 27.78 0.073 0.080 0.1118 0.039170 24.31 0.07 23.77 1204.82
z 0.87 0.76 19.60 1687.9 27.39 0.051 0.055 0.0806 0.039292 24.15 0.05 23.21 1412.44
y 0.85 0.75 18.63 2140.8 26.56 0.024 0.029 0.1992 0.039351 23.72 0.04 22.28 1541.91


In [31]:
filterlist = ('u', 'g', 'r', 'i', 'z', 'y')
t2['dm5_X1.2'] = {}
for f in filterlist:
    t2['dm5_X1.2'][f] = t2['m5'][f] - t2_12['m5'][f]
t2['dm5_X1.2']

{'g': 0.15834095414404459,
 'i': 0.13304235821886223,
 'r': 0.14203109058798091,
 'u': 0.21490542510367661,
 'y': 0.14081488123698449,
 'z': 0.1297758290595219}

In [32]:
tau = 60.0/30.0
t2['dcm_60'] = {}
for f in filterlist:
    t2['dcm_60'][f] = t2['dCm_infinity'][f] - 1.25 * np.log10(1 + (np.power(10, 0.8*t2['dCm_infinity'][f]) - 1) / (tau))

In [33]:
t2['dm_fid'] = {}
t2['dm_min'] = {}
t2['m5_fid'] = {'u': 23.9, 'g': 25.0, 'r': 24.7, 'i': 24.0, 'z': 23.3, 'y': 22.1}
t2['m5_min'] = {'u': 23.4, 'g': 24.6, 'r': 24.3, 'i': 23.6, 'z': 22.9, 'y': 21.7}
for f in filterlist:
    t2['dm_fid'][f] = t2['m5'][f] - t2['m5_fid'][f]
    t2['dm_min'][f] = t2['m5'][f] - t2['m5_min'][f]

In [50]:
l = ['skyMag', 'FWHMeff', 'gamma', 'kAtm', 'Cm', 'dCm_infinity', 'dcm_60', 'm5', 'dm5_X1.2',  
     'm5_fid', 'dm_fid', 'm5_min', 'dm_min']

writestring = '  \t\t& $u$\t& $g$\t & $r$\t& $i$\t& $z$\t& $y$\t \\\\'
print(writestring)
for li in l:
    if li == 'dCm_infinity':
        writestring = 'dCm_inf' + '\t\t& '
    else:
        writestring = li + '\t\t& '
    if li == 'gamma' or li == 'kAtm':
        mat = '%.3f\t& '
    else:
        mat = '%.2f\t& '
    for f in filterlist:
        writestring += mat % t2[li][f]
    writestring = writestring[:-2] + ' \\\\'
    print(writestring)

  		& $u$	& $g$	 & $r$	& $i$	& $z$	& $y$	 \\
skyMag		& 22.95	& 22.24	& 21.20	& 20.47	& 19.60	& 18.63	 \\
FWHMeff		& 0.93	& 0.87	& 0.83	& 0.80	& 0.78	& 0.76	 \\
gamma		& 0.038	& 0.039	& 0.039	& 0.039	& 0.039	& 0.039	 \\
kAtm		& 0.504	& 0.210	& 0.126	& 0.096	& 0.068	& 0.176	 \\
Cm		& 22.74	& 24.38	& 24.43	& 24.30	& 24.15	& 23.70	 \\
dCm_inf		& 0.75	& 0.19	& 0.10	& 0.07	& 0.05	& 0.04	 \\
dcm_60		& 0.25	& 0.08	& 0.05	& 0.04	& 0.02	& 0.02	 \\
m5		& 23.40	& 24.76	& 24.34	& 23.90	& 23.34	& 22.43	 \\
dm5_X1.2		& 0.21	& 0.16	& 0.14	& 0.13	& 0.13	& 0.14	 \\
m5_fid		& 23.90	& 25.00	& 24.70	& 24.00	& 23.30	& 22.10	 \\
dm_fid		& -0.50	& -0.24	& -0.36	& -0.10	& 0.04	& 0.33	 \\
m5_min		& 23.40	& 24.60	& 24.30	& 23.60	& 22.90	& 21.70	 \\
dm_min		& 0.00	& 0.16	& 0.04	& 0.30	& 0.44	& 0.73	 \\
